In [23]:
import numpy as np
import pickle
import os

In [24]:
sampling_rate = 16 #Hz
smooth_factor = 0.8

In [25]:
def resample(data, rate):
    interval = 1/rate
    duration = data[-1, 0]
    ts = np.arange(0, duration, interval)    
    count = len(ts)
    #print(interval, ", ",count)
    res = np.zeros((count-1, data.shape[1]))
    
    i = 0
    j = 0
    while i < count-1:
        t = ts[i]
        res[i, 0] = t        
        t1 = data[j, 0]
        t2 = data[j+1, 0]
        
        if t1 <= t < t2:
            factor = (t - t1)/(t2-t1);
            res[i, 1:] = (1-factor)*data[j, 1:]  + factor*data[j+1, 1:]
            i+=1
        
        while not(data[j, 0] <= ts[i] < data[j+1, 0]):
            j+=1   
            
    return res

In [26]:
def smooth_data(data, factor):
    count = len(data)
    for i in range(1, count):
        data[i, :] = factor*data[i-1, :] + (1-factor)*data[i, :]    
    return data

In [27]:
def time_to_index(data, ts):
    count = len(ts)
    ix = np.zeros(ts.shape)
    
    i = 0
    j = 0
    while i < count:        
        if data[j, 0] <= ts[i] < data[j+1, 0]:
            ix[i] = j
            i += 1
        
        if i==count:
            break;
        
        while not(data[j, 0] <= ts[i] < data[j+1, 0]):
            j += 1
            
    return ix        
    

In [28]:
def process_annots_uva(annots, accel):
    annots[:, 0] = time_to_index(accel, annots[:, 0])
    
    for i in range(len(annots)):
        if 1 <= annots[i, 1] < 400:
            annots[i, 1] = 1
        elif 400 <= annots[i, 1] < 1000:
            annots[i, 1] = 2
        else:
            print("XXXXXXX UVA annot Problem XXXXXXXXXXX")
            
    annots = annots.astype(int)
    return annots
    

In [29]:
def get_data():
    path = 'C:\\ASM\\DevData\\eating\\python\\our_lab_data.pkl'
    if os.path.exists(path):
        print("Uva lab data file exists.")
        file = open(path, "rb")        
        data = pickle.load(file)
        file.close()
        return data
    

In [30]:
def read_data_uva_lab():
    print("Reading data UVA lab")
    path = 'C:\\ASM\\DevData\\eating\\our_data\\lab_data\\'
    data = []
    for i in range(35):        
        accel = np.genfromtxt(path + "accel_"+str(i), delimiter=',')        
        print("\n\nSession ", str(i), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        #print(accel[:10, :])
        
        accel = resample(accel, sampling_rate)
        #print(accel[:10, :])
        accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
        print("Session ", str(i), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        print(accel[:10, :])
          
        annots = np.genfromtxt(path+"annot_"+str(i), delimiter=',') 
        print("Session ", str(i), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
        annots = process_annots_uva(annots, accel)        
        print(annots[:10, :])
        
        dsess = [accel, annots]        
        dsubject = [dsess]        
        data.append(dsubject)
    
    path = 'C:\\ASM\\DevData\\eating\\python\\our_lab_data.pkl'
    with open(path, "wb") as file:
        pickle.dump(data, file)
        
    return data
    

In [ ]:
data = read_data_uva_lab()
print(len(data))

In [33]:
def read_data_uva_free():
    print("Reading data UVA free")
    path = 'C:\\ASM\\DevData\\eating\\our_data\\free_data\\'
    data = []
    for i in range(16):        
        accel = np.genfromtxt(path + "accel_"+str(i), delimiter=',')                
        print("Session ", str(i), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        #print(accel[:10, :])
        
        accel = resample(accel, sampling_rate)
        #print(accel[:10, :])
        accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
        print("Session ", str(i), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        print(accel[:10, :])
          
        annots = [] 
        
        dsess = [accel, annots]        
        dsubject = [dsess]        
        data.append(dsubject)
    
    path = 'C:\\ASM\\DevData\\eating\\python\\our_free_data.pkl'
    with open(path, "wb") as file:
        pickle.dump(data, file)
        
    return data

In [ ]:
data = read_data_uva_free()
print(len(data))

In [42]:
def process_bite_label_steven(label):
    label = label.replace("P","1").replace("D","2").replace("I","3").replace("Q","4")
    label = label.replace("N","5").replace("C","6").replace("S","7").replace("M","8")
    label = label.replace("R","1").replace("L","2").replace("X","-1")
    return int(label)

def process_bite_annots_steven(annots, accel):    
    a = np.zeros((len(annots), 2))
    for i in range(len(annots)):
        s = annots[i].split(",")
        t1 = float(s[0].rstrip())
        t2 = float(s[1].rstrip())
        label = process_bite_label_steven(s[2].rstrip())
        a[i, 0] = t1+t2/2
        a[i, 1] = label
    
    res = []
    hand = 0
    i = 0
    while i <len(a)-1:
        #print(a[i, 0], ", ", a[i, 1])
        if a[i, 1]==22 and a[i+1, 1]==21:
            hand = 3
            i+=1
        elif a[i, 1]==21:
            hand = 1
        elif a[i, 1]==22:
            hand = 2
        elif a[i, 1]==3 or a[i, 1]==4:
            if hand==0:
                print("********** Hand is not found : " , annots[i] , ", line no: ", (i + 1));
                i+=1
                continue
            
            r = [0, 0]
            r[0] = a[i, 0]
            if a[i, 1]==3:
                r[1] = 1
            else:
                r[1] = 2
            
            if hand ==1 or hand ==3:
                res.append(r)
        i+=1
        
    #print(res)            
    res = np.array(res)
    #print(res.shape)
    res[:, 0] = time_to_index(accel, res[:, 0])
    res = res.astype(int)
    return res

ERROR! Session/line number was not unique in database. History logging moved to new session 47


In [45]:
def read_data_steven_lab():
    path = 'C:\\ASM\\PublicData\\ACE_lab_public_2\\lab_public\\'
    
    data = [];
    for subject in range(7):
        dsubject = [];
        for sess in range(2):
            if (subject==0 and sess==0) or (subject==1 and sess==1):
                continue
            
            filePathAccel = path + "\\0" + str(subject) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
            filePathAnnots = path + "\\0" + str(subject) + "\\000" + str(sess) + "\\annot_events.csv";
            
            accel = np.genfromtxt(filePathAccel, delimiter=',')
            accel = accel[:, :4]
            accel[:, 0] = accel[:, 0]/1e9
            print("Subject ", str(subject), ", Session ", str(sess), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            print(accel[:10, :])
        
            accel = resample(accel, sampling_rate)            
            accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
            print("Subject ", str(subject), ", Session ", str(sess), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            print(accel[:10, :])
          
            file = open(filePathAnnots)
            annots = file.readlines()
            annots = process_bite_annots_steven(annots, accel)        
            print("Subject ", str(subject), ", Session ", str(sess), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
            print(annots[:10, :])
            
            dsess = [accel, annots]        
            dsubject.append(dsess)        
            
        data.append(dsubject)
    
    path = 'C:\\ASM\\DevData\\eating\\python\\steven_lab_data.pkl'
    with open(path, "wb") as file:
        pickle.dump(data, file)
    
    return data
    
    

In [ ]:
data = read_data_steven_lab()
print(len(data))

In [99]:
def get_meal_code(meal):    
    code = -1
    if meal == "meal" or meal=="lunch" or meal=="dinner":
        code = 1
    elif meal=="snack":
        code = 2    
    elif meal=="drink":
        code = 3
    return code;

def process_meal_annots_steven(annots, accel, subject_code, session):
    meal_count = len(annots)
    accel_count = len(accel)
    if subject_code == 5 and session == 0:
        meal_count = 9
    elif subject_code == 6 and session == 0:
        meal_count = 2
    elif subject_code == 102 and session == 1:
        meal_count = 1
    elif subject_code == 104 and session == 0:
        meal_count = 7
    
    j = 0
    a = np.zeros((meal_count, 3))
    for i in range(meal_count):        
        s = annots[i].split(",")
        t1 = float(s[1].rstrip())
        t2 = float(s[2].rstrip())
        a[i, 2] = get_meal_code(s[3].rstrip().strip())
        
        if a[i, 2]==-1:
            print(" \n\n******************* Meal code problem *************** code is -1\n\n")
        
        t = t1
        while j<accel_count-1:
            if accel[j, 0] <= t < accel[j+1, 0]:            
                a[i, 0] = j
                break            
            j+=1
            
        t = t2
        while j<accel_count-1:
            if accel[j, 0] <= t < accel[j+1, 0]:            
                a[i, 1] = j
                break            
            j+=1
        
        if a[i][1] == 0:
            if subject_code == 107 and session == 0 or subject_code == 107 and session == 2:
                a[i, 1] = accel_count - 1
        
        if a[i, 0] == 0 or a[i, 1] == 0:
            print("***** Time indexing problem. Meal line: " , (i + 1))
        
    return a

In [102]:
def read_data_steven_free():
    path = 'C:\\ASM\\PublicData\\PHI_ACE-E-FL_public\\'
    subject_codes = [2, 3, 4, 5, 6, 101, 102, 103, 104, 107, 109]
    
    data = [];
    for subject in range(len(subject_codes)):
        dsubject = [];
        sess_count = 2
        if subject_codes[subject] == 107:
            sess_count = 5
        
        for sess in range(sess_count):
            if subject_codes[subject] < 10:
                filePathAccel = path + "\\0" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
                filePathAnnots = path + "\\0" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\meal_events.csv";
            elif subject_codes[subject] == 109:
                filePathAccel = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess+3) + "\\watch_right_000" + str(sess+3) + ".csv";
                filePathAnnots = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess+3) + "\\meal_events.csv";
            else:
                filePathAccel = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
                filePathAnnots = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\meal_events.csv";
          
            accel = np.genfromtxt(filePathAccel, delimiter=',')
            accel = accel[:, :4]
            accel[:, 0] = accel[:, 0]/1e9
            print("\n\nSubject ", str(subject), ", Session ", str(sess), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            #print(accel[:10, :])
        
            accel = resample(accel, sampling_rate)            
            accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
            print("Subject ", str(subject), ", Session ", str(sess), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            #print(accel[:10, :])
          
            file = open(filePathAnnots)
            annots = file.readlines()
            annots = process_meal_annots_steven(annots, accel, subject_codes[subject], sess)        
            print("Subject ", str(subject), ", Session ", str(sess), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
            print(annots)
            
            dsess = [accel, annots]        
            dsubject.append(dsess)        
            
        data.append(dsubject)
    
    path = 'C:\\ASM\\DevData\\eating\\python\\steven_free_data.pkl'
    with open(path, "wb") as file:
        pickle.dump(data, file)
    
    return data
    

In [103]:
data = read_data_steven_free()
print(len(data))



Subject  0 , Session  0  Before process >> Duration:  0.0  -  40424.5294861 , Count: 725551
Subject  0 , Session  0  After process >> Duration:  0.0  -  40424.4375 , Count: 646792
Subject  0 , Session  0  Annots >> Duration:  12888.0  -  294585.0 , Count: 3
[[  1.28880000e+04   1.88310000e+04   1.00000000e+00]
 [  7.72310000e+04   7.81750000e+04   3.00000000e+00]
 [  2.94585000e+05   3.22326000e+05   1.00000000e+00]]


Subject  0 , Session  1  Before process >> Duration:  0.0  -  38185.6452497 , Count: 550555
Subject  0 , Session  1  After process >> Duration:  0.0  -  38185.5625 , Count: 610970
Subject  0 , Session  1  Annots >> Duration:  33449.0  -  479052.0 , Count: 5
[[  3.34490000e+04   4.43780000e+04   1.00000000e+00]
 [  1.32596000e+05   1.33317000e+05   3.00000000e+00]
 [  3.10975000e+05   3.21961000e+05   1.00000000e+00]
 [  3.99570000e+05   4.00600000e+05   3.00000000e+00]
 [  4.79052000e+05   4.79898000e+05   3.00000000e+00]]


Subject  1 , Session  0  Before process >> D